# Preprocessing
 > **1. CUDA Setup**. --> Look at **README.md**
 
 > **2. Yolov5 Setup** --> **setup.ipynb**
 
 > **3. Preparing data**  --> For data preprocessing look at **data_preprocessing.ipynb** notebook.

# Import libraries

In [ ]:
import torch
from yolov5 import utils
import torch
from IPython import display
from IPython.display import clear_output
from pathlib import Path
import yaml
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import io
import os
import cv2
import json
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline

# Initialize Static Hyperparameters.

In [ ]:
PROJECT_NAME = "yolov5_train"
BASE_MODEL = "yolov5m6.pt"
TRAIN_BATCH = 32
TRAIN_EPOCHS = 200
VAL_BATCH = 64

In [ ]:
IMAGES_PATH = "yolov5/data/train/images"
LABELS_PATH = "yolov5/data/train/labels"
NOTES_PATH = "yolov5/data/data.json"

# Train.

In [ ]:
# Delete old results if exists
wildcard = f"{PROJECT_NAME}/feature_extraction*"
! rm -r $wildcard

In [ ]:
! python3.10 yolov5/train.py --batch $TRAIN_BATCH --epochs $TRAIN_EPOCHS --data "data.yaml" --weights $BASE_MODEL --project $PROJECT_NAME --name 'feature_extraction' --cache --freeze 12

# Validation

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/validation_on_test_data*"
! rm -r $wildcard

In [ ]:
WEIGHTS_BEST = f"{PROJECT_NAME}/feature_extraction/weights/best.pt"
! python3.10 yolov5/val.py --weights $WEIGHTS_BEST --batch $VAL_BATCH --data 'data.yaml' --task test --project $PROJECT_NAME --name 'validation_on_test_data' --augment

# Test.

In [ ]:
# Delete old results
wildcard = f"{PROJECT_NAME}/detect_test*"
! rm -r $wildcard

In [ ]:
! python3.10 yolov5/detect.py --weights $WEIGHTS_BEST --conf 0.6 --source 'yolov5/data/test/images' --project $PROJECT_NAME --name 'detect_test' --augment --line=3

# Inference

In [ ]:
WEIGHTS_BEST = f"{PROJECT_NAME}/feature_extraction/weights/best.pt"
!python3.10 yolov5/detect.py --source 'yolov5/data/test/images' --weights $WEIGHTS_BEST --conf-thres 0.4

In [ ]:
def read_images(dirpath):
    images = []
    for img_filename in os.listdir(dirpath):
        if img_filename.endswith((".jpg", ".png")):
            images.append(mpimg.imread(f"{dirpath}/{img_filename}"))
    return images

In [ ]:
def label_test_images(test_images_path, test_labels_path, classes):
    test_images = os.listdir(test_images_path)
    labeled_images = []

    for idx, test_image_filename in enumerate(test_images):
        image = mpimg.imread(f"{test_images_path}/{test_image_filename}")

        x_shape, y_shape = image.shape[1], image.shape[0]

        test_label_filename = f"{test_image_filename[:-4]}.txt"

        with open(f"{test_labels_path}/{test_label_filename}", "r") as f:
            lines = f.readlines()

            for line in lines:
                # Parse line
                box = line.split()
                class_idx = box[0]

                class_name = names[int(class_idx)]
                x_center, y_center, box_w, box_h = int(float(box[1])*x_shape), int(float(box[2])*y_shape), int(float(box[3])*x_shape), int(float(box[3])*y_shape)
                x1, y1, x2, y2 = x_center-int(box_w/2), y_center-int(box_h/2), x_center+int(box_w/2), y_center+int(box_h/2)

                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 3)
                cv2.putText(image, class_name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 3)

        labeled_images.append(image)

    return labeled_images


In [ ]:
yaml_file_path = "yolov5/data/data.yaml"
names = []
with open(os.path.join(os.getcwd(), yaml_file_path), 'r') as file:
    try:
        data = yaml.safe_load(file)
        names.extend(data["names"])
        print("YAML file content:", data)
    except yaml.YAMLError as e:
        print("Error reading YAML file:", e)
        
print("names", names)

In [ ]:
detect_path = f"{PROJECT_NAME}/detect_test"
test_images_path = f"yolov5/data/test/images"
test_labels_path = f"yolov5/data/test/labels"

detected_images = read_images(detect_path)
test_labeled_images = label_test_images(test_images_path, test_labels_path, classes=names)

stacked_images = [np.hstack([detected_images[idx], test_labeled_images[idx]]) for idx in range(len(detected_images))]

In [ ]:
for image in stacked_images:
    fig = plt.figure(figsize=(40, 15))
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(image)

# Save model

>  - **PROJECT_FOLDER{yolov5_train}/feature_extraction/weights/best.pt**

>  - **best.pt** will be used to load it in your project to predict.